## Bilinear-Biquadratic Chain

$$
H=J\sum_i \cos\theta \left( \boldsymbol S_i\cdot \boldsymbol S_{i+1}\right) + \sin\theta\left(\boldsymbol S_i\cdot \boldsymbol S_{i+1}\right)^2
$$

In [ ]:
using MPSKitModels, MPSKit, TensorKit, TensorOperations, Plots

include("hamiltonian.jl")
include("plot_spectrum.jl")

using Logging
disable_logging(LogLevel(Info))

## Entanglement Spectra

Schmidt decomposition:
$$
\ket \Psi = \sum_{\alpha=1}^D\lambda_\alpha \ket \Phi^A_\alpha \otimes \ket \Phi^B_\alpha
$$
The Schmidt coefficients resemble entanglement. High coefficients mean more entanglement whilst $\lambda_\alpha=\{1, 0, 0, ...\}$ means we have a product state.

We can approximate the state by truncating the MPS when the Schmidt coefficients are sufficiently low.

#### Ferromagnetic spin=1:
- product state

In [ ]:
ham = bilinear_biquadratic_hamiltonian(spin=1, θ=π)
Ψ = optimize_groundstate(ham, bond=10, maxiter=500)
entanglementplot(Ψ)

#### AntiFerromagnetic spin=1:
- in SPT phase
- not a product state

In [ ]:
ham = bilinear_biquadratic_hamiltonian(spin=1, θ=0)
Ψ = optimize_groundstate(ham, bond=12, maxiter=500)
entanglementplot(Ψ)

#### AKLT:
- $\theta = \arctan(1/3)$
- exactly represented by $\textrm{bond}=2$

In [ ]:
ham = bilinear_biquadratic_hamiltonian(spin=1, θ=atan(1/3))
Ψ = optimize_groundstate(ham, bond=6, maxiter=500)
entanglementplot(Ψ)

## SPT

Choose a state with a symmetry group $\mathrm G$.

The *Fundamental theorem of MPS* $\implies$ different equivalence classes.

Two states of the same 'class' can be connected via a path of gapped parent Hamiltonians, thus no phase transitions $\implies$ class = phase.

If we travel from one class to another we must cross a phase transition if the symmetry is preserved $\implies$ symmetry protected.

The amount of equivalence classes is equal to the order of the second cohomology group $H^2(\mathrm G, \mathrm U_1)$.

Symmetry has to be represented by a **physical linear rep.** but can be represented by a **virtual projective rep.**


#### AntiFerromagnet spin=1

$H^2(\mathrm{SO}_3, \mathrm U_1)=\mathbb Z_2\implies$ 2 phases:
- Trivial phase: **linear rep.**
- Haldane phase: **projective rep.**

The half integer spin reps. are projective reps. of $\mathrm{SO}_3$. The Haldane phase thus has half integer reps.

The right fixed point $\rho$ is invariant under our virtual symmetry:
$$
X_g\rho X_g^\dagger = \rho \implies \textrm{$\rho$ is proportional to }
$$
The degeneracy of the eigenvalues of $\rho$, which are the Schmidt coefficients, are determined by the irreps. of $X_g$.

## Degeneracies

These degeneracies are always even for our spin=1 $\mathrm{SO}_3$ state as the **virtual projective reps.** are the half integer reps. which have even dimension.

In [ ]:
spectrum = plot_entanglement(spin=1, θ=0, bond=12)
annot(spectrum, 2, "spin=1/2")
annot(spectrum, 6, "spin=3/2")
annot(spectrum, 8, "spin=1/2")
display(current())

## Convergence

If we optimize the HAFM for $\textrm{bond}=7$ we get poor convergence, whilst $\textrm{bond}=8$ gets good convergence.

The state is approximated by a direct sum of irreps and cutting one of these irreps in half makes for a harder approximation.

In [ ]:
ham1 = bilinear_biquadratic_hamiltonian(spin=1, θ=0.0)
Ψ1 = optimize_groundstate(ham1, bond=7)
entanglement_spectrum(Ψ1)

In [ ]:
ham2 = bilinear_biquadratic_hamiltonian(spin=1, θ=0.0)
Ψ2 = optimize_groundstate(ham1, bond=8)
entanglement_spectrum(Ψ2)

## Approaching AKLT

$\theta\to \arctan(1/3)\approx 0.3218$

For increasing $\theta$ we get an increase in the excitation energy of the magnon.


In [ ]:
θ_range = linrange(0, atan(1/3), 20; endpoint=false) # approach AKLT
# θ_range = linrange(0, π/4, 30; endpoint=false) # go past AKLT
spectra = spectrum_approach(θ_range, spin=1, bond=12, maxiter=1000)

display(current())

We take the Schmidt value $\chi=4$ and see how it changes for each $\theta$:

In [ ]:
schmidt_range(θ_range, spectra, 3)
vline!([atan(1/3)], label=nothing)
display(current())

In [ ]:
ham = bilinear_biquadratic_hamiltonian(spin=1, θ=0.024*π)
Ψ = optimize_groundstate(ham, bond=8, maxiter=500)
entanglementplot(Ψ)

Notes:
- It has been shown that the Haldane gap is only invariant under physical linear representations (not sure if this is always true for SPT phases)
- The order of the virtual irreps in the SPT phase is just what happens to make the ground state the lowest energy
- Excitation gap of the magnon becomes zero as $\theta \to \pi/4$